## From Twitter import Sanders's tweets and save it to tsv file

In [ ]:
#!/usr/bin/env python3
from twitter import *
import json

consumer_key = "vAm5pdHyfy1xbTdwGtTP1A"
consumer_secret = "7Kp9IIvYbseJzpknZPOXjcXQHfXsSG4hLUFZ9iz1Ws"
access_key = "1448875278-tFuv7p6heGXhCk5xfDQ0Btqh5sFkVkRJbodOUyI"
access_secret = "SHm0LQtrwzHgpqkudPwXIJWpzZdvUAGTwBq5HSlxOzYFF"

tweetidfilename='sanders.txt'

auth = OAuth(access_key, access_secret, consumer_key, consumer_secret)
twitter = Twitter(auth = auth)

tweetids=[e.strip() for e in open(tweetidfilename).readlines()]

# LIMIT TO 900 TWEETS B/C OF API RESTRICTIONS AND CREATE CHUNKS OF 100
def chunker(seq, size):
    return [seq[pos:pos + size] for pos in range(0, len(seq), size)]
batches = chunker(tweetids[:900],100)

# API expects comma-separated strings, no lists:
batches = [','.join(l) for l in batches]

posts = []

for batch in batches:
    posts += twitter.statuses.lookup(_id=batch)


headers=["text","retweeted","retweet_count","hashtags","urls","created_at","description","screen_name","name","friends_count","followers_count","statuses_count"]
with open('sanders.tsv',mode='w') as fo:
        fo.write("\t".join(headers)+"\n")
        for post in posts:
            output=[]
            output.append(post["text"].replace("\t"," ").replace("\n"," ").replace("\r"," "))
            output.append(str(post["retweeted"]))
            output.append(str(post["retweet_count"]))            
            hashtags = [tag["text"] for tag in post["entities"]["hashtags"]]        
            output.append(str(hashtags))
            urls = [u["expanded_url"] for u in post["entities"]["urls"]]    
            output.append(str(urls))    
            output.append(post["created_at"])
            output.append(post["user"]["description"].replace("\t"," ").replace("\n"," ").replace("\r"," "))
            output.append(post["user"]["screen_name"])
            output.append(post["user"]["name"].replace("\t"," ").replace("\n"," ").replace("\r"," "))
            output.append(str(post["user"]["friends_count"]))
            output.append(str(post["user"]["followers_count"]))
            output.append(str(post["user"]["statuses_count"]))            
            fo.write("\t".join(output)+"\n")


## open the file and clear the list with replace()

In [ ]:
import csv
tweetlist = []
processedlist = []

with open('/Users/shujun/Code/Python/Sentiment/Dictionary_based/Sanders.tsv')as fi:
    reader = csv.reader(fi, delimiter='\t')## rade the contents into a data frame
    next(reader)##skip the header row with the next()
    for row in reader:
        tweet = row[0]# list
        tweet_processed=tweet.lower().replace("!"," ").replace("."," ").replace("?"," ").replace("'"," ").replace('"'," ").replace("#"," ").replace(':'," ")
        tweetlist.append(tweet)
        processedlist.append(tweet_processed)

In [ ]:
from pprint import pprint
pprint(tweetlist)

## Import positivelist and negativelist

In [ ]:
positivelist = open("/Users/shujun/Code/Python/Sentiment/Dictionary_based/positive-words.txt").read().splitlines()
print(positivelist)

In [ ]:
negativelist = open("/Users/shujun/Code/Python/Sentiment/negative-words.txt", encoding = "ISO-8859-1").read().splitlines()

## Real analysis

In [58]:
negcountlist = []
poscountlist = []
sentilist = []

for tweet in processedlist:
    poscount = 0
    negcount = 0
#     print("Analyzing this one:",tweet) #tweet is a string here
    
    for word in tweet.split(): ##split a string into a list
        if word in positivelist:
            poscount+=1
        elif word in negativelist:
            negcount+=1
            
#     print("It contains", poscount, "positive words and", negcount, "negative words.")
#     print("--------")
    
    poscountlist.append(poscount)
    negcountlist.append(negcount)
    sentilist.append((poscount-negcount)/len(tweet.split()))
print("Average sentiment:",sum(sentilist)/len(sentilist))

Average sentiment: 0.010680186833531322


note: 
next(): skip the header row
splitlines(): splits a string into a list. The splitting is done at line breaks
split(): splits a string into a list.You can specify the separator, default separator is any whitespace.


In [61]:
from nltk.sentiment import vader
senti=vader.SentimentIntensityAnalyzer()
compoundlist = []
for tweet in tweetlist:
    compoundlist.append(senti.polarity_scores(tweet)['compound'])

In [ ]:
print(compoundlist)

In [63]:
print(sum(compoundlist)/len(compoundlist))

0.12155276816608983
